In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [1]:
import os
import numpy as np 
import csv
from glob import glob

In [2]:
import pandas as pd  
import matplotlib.pyplot as plt  
%matplotlib inline

from sklearn.model_selection import train_test_split  
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error

In [24]:
ROOT_DIR = os.path.abspath("../../")
DS_DIR = os.path.join(ROOT_DIR, "Datasets/awe")


### Model generation using Imagenet weights

In [ ]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()

### Image Feature Extraction

In [79]:
def getVector():
    return np.array([1,2,3,5,6,7,8,9,10])


In [114]:
vgg16_feature_list = []
for (dirpath, u_dirnames, filenames) in os.walk(DS_DIR):
    dirnames = sorted(u_dirnames)
    for i in range(0,len(dirnames)):
        FILE_DIR = os.path.join(DS_DIR, str(dirnames[i]))
        files = glob(FILE_DIR+'/*.png') 
        for j in range (0,len(files)):
            #print(files[j])
            image_class = np.array([int(str(dirnames[i]))])
            #img = image.load_img(files[j], target_size=(224, 224))
            #img_data = image.img_to_array(img)
            #img_data = np.expand_dims(img_data, axis=0)
            #img_data = preprocess_input(img_data)
            #vgg16_feature = model.predict(img_data)
            #image_vector = np.array(vgg16_feature)            
            image_vector = getVector()                       
            vgg16_feature_np = np.concatenate((image_class, image_vector), axis=0)                                  
            vgg16_feature_list.append(vgg16_feature_np.flatten())            
    break

#print(len(vgg16_feature_list))
#print(len(vgg16_feature_np))
head = []
head.append('class')
for i in range(1, len(vgg16_feature_np)):
    head.append('v'+str(i))

head_np = []
head_np.append(head)
#print(head_np)

1000
10
[['class', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9']]


### Build CSV file

In [115]:
with open('vgg16.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(head_np)
    writer.writerows(vgg16_feature_list)
csvFile.close()

### Get Distance metric

In [3]:
def chiSquared(a,b):
    res = np.sqrt(0.5*np.sum((a-b)**2/(a+b+1e-6)))
    return res

def chiSquared2(a,b):
    #res = np.sqrt(0.5*np.sum((a-b)**2/(a+b+1e-6)))
    return 0.5*np.sum((a-b)**2/(a+b+1e-6))

def getMetrics(csvfilename):
    data = pd.read_csv(csvfilename)
    X = data.drop('class', axis=1)  
    y = data['class']
    y_pred = np.zeros(data.shape)
    for i in range(len(X)):
        #minimun = 1e+10;    
        for j in range (len(X)):
            res = chiSquared(X.values[i], X.values[j])
            #if res < minimun and i != j:
                #minimun = res
            y_pred[i][j] = y[j] 
                
    count = 0
    for i in range (len(y)):
        if y[i] == y_pred[i]:
            count = count + 1    
    msq = np.sqrt(mean_squared_error(y, y_pred))
    rank1 = count*100/len(y)
    res = 'MSR: '+ str(msq) + ' - Rank1: '+ str(rank1)    
    return res

In [ ]:
print(getMetrics("calculated_features/awe-awet_bsif.csv"))

In [137]:
print(getMetrics("calculated_features/awe-awet_dsift.csv"))

MSR: 32.4314970361 - Rank1: 39.3


In [4]:
print(getMetrics("calculated_features/awe-awet_hog.csv"))

1000


In [ ]:
print(getMetrics("calculated_features/awe-awet_lbp.csv"))

In [ ]:
print(getMetrics("calculated_features/awe-awet_lpq.csv"))

In [ ]:
print(getMetrics("calculated_features/awe-awet_rilpq.csv"))

In [ ]:
print(getMetrics("calculated_features/awe-awet_poem.csv"))

In [134]:
print(getMetrics("calculated_features/vgg16.csv"))

MSR: 57.3018324314 - Rank1: 1.0


In [30]:
csvfilename = "calculated_features/awe-awet_lbp.csv"
data = pd.read_csv(csvfilename)
X = data.drop('class', axis=1)  
y = data['class']
s = (len(X), len(X), 2)
y_pred = np.zeros(s)
for i in range(len(X)):
    #minimun = 1e+10;    
    for j in range (len(X)):
        res = chiSquared(X.values[i], X.values[j])
        #if res < minimun and i != j:
            #minimun = res
        y_pred[i][j] = [res, y[j]]

In [31]:
len(y_pred)

1000

In [61]:
y_pred[35][:45]

array([[   0.        ,    1.        ],
       [   1.        ,  110.47814114],
       [   1.        ,  110.61641101],
       [   1.        ,  110.6996499 ],
       [   1.        ,  111.18752886],
       [   1.        ,  111.46490242],
       [   1.        ,  111.66306437],
       [   1.        ,  111.67152393],
       [   1.        ,  111.8364342 ],
       [   1.        ,  111.91574444],
       [   2.        ,  111.9564981 ],
       [   2.        ,  111.97844336],
       [   2.        ,  112.06293026],
       [   2.        ,  112.19035266],
       [   2.        ,  112.3526495 ],
       [   2.        ,  112.50500309],
       [   2.        ,  112.71699069],
       [   2.        ,  112.75911206],
       [   2.        ,  113.04783061],
       [   2.        ,  113.48304693],
       [   3.        ,  113.53347571],
       [   3.        ,  113.72653878],
       [   3.        ,  113.7371122 ],
       [   3.        ,  113.90114861],
       [   3.        ,  113.91107164],
       [   3.        ,  1

In [55]:
y_pred.shape

(1000, 1000, 2)

In [59]:
y_gg = y_pred

In [57]:
for i in range(len(y_gg)):
    y_gg[i].sort(axis=0) 

In [60]:
y_gg[35][:45]

array([[   0.        ,    1.        ],
       [   1.        ,  110.47814114],
       [   1.        ,  110.61641101],
       [   1.        ,  110.6996499 ],
       [   1.        ,  111.18752886],
       [   1.        ,  111.46490242],
       [   1.        ,  111.66306437],
       [   1.        ,  111.67152393],
       [   1.        ,  111.8364342 ],
       [   1.        ,  111.91574444],
       [   2.        ,  111.9564981 ],
       [   2.        ,  111.97844336],
       [   2.        ,  112.06293026],
       [   2.        ,  112.19035266],
       [   2.        ,  112.3526495 ],
       [   2.        ,  112.50500309],
       [   2.        ,  112.71699069],
       [   2.        ,  112.75911206],
       [   2.        ,  113.04783061],
       [   2.        ,  113.48304693],
       [   3.        ,  113.53347571],
       [   3.        ,  113.72653878],
       [   3.        ,  113.7371122 ],
       [   3.        ,  113.90114861],
       [   3.        ,  113.91107164],
       [   3.        ,  1